In [1]:
import numpy as np
from datetime import datetime
import ROOT
import math
import h5py
import xml.etree.ElementTree as ET
import os
import json
import argparse
import sys
phi_res = 128/(2*math.pi)

# --------------------------------------------------------------
# Constants
# --------------------------------------------------------------
# Maximum numbers of each object
NUM_JETS = 10
NUM_ELECTRONS = 6
NUM_MUONS = 6
NUM_PHOTONS = 6
NUM_L1 = 6

# Min/max pt and eta values
MIN_JET_PT = 0
MIN_PHELMU_PT = 0
MAX_ETA = 1000

# --------------------------------------------------------------
# Trigger Menu Stuff
# --------------------------------------------------------------
def get_hlt_chains_with_prescale_one(filename):
    # Open and load the JSON data
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # Filter the entries that have "prescaleCombined" = 1
    result = []
    for item in data:
        if item.get('prescaleCombined') == 1:
            result.append(item['hltChain'])
    
    return result

def get_L1_chains_with_prescale_one(filename):
    # Open and load the JSON data
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # Filter the entries that have "prescaleCombined" = 1
    result = []
    for item in data:
        if item.get('prescaleCombined') == 1:
            result.append(item['l1Item'])
    
    return np.unique(result)

HLT_trig_names = get_hlt_chains_with_prescale_one('/eos/home-m/mmcohen/gelato/trigger_menus/CombinedPrescale_l1_230_hlt_412.json')
L1_trig_names = get_L1_chains_with_prescale_one('/eos/home-m/mmcohen/gelato/trigger_menus/CombinedPrescale_l1_230_hlt_412.json')



# --------------------------------------------------------------
# Functions that read the TTree and make the data for HLTAD and L1AD
# --------------------------------------------------------------
def sort_and_pad(in_data, max_objects):
    """input an object of size (4, num_objects), as well as the maximum number of objects wanted.
    in_data[0] corresponds to the pt array, and the other three are E, eta, phi."""
    
    indices = np.argsort(in_data[0])[-max_objects:][::-1] # indices that sort the in_data by pt
    arrs = []
    for array in in_data:

        # Sorting
        sorted_arr = array[indices]

        # Padding
        if len(array) < max_objects:
            sorted_arr = np.concatenate((sorted_arr, np.zeros(max_objects - len(array))))

        arrs.append(sorted_arr)
        
    
    arrs = np.array(arrs).T
    return np.array(arrs)

def make_mask(min_pt, max_eta, pt, eta):
    """Make a mask on pt and eta of event, in case I want to make any data cuts."""

    return [(pt > MIN_JET_PT) and (abs(eta) < MAX_ETA) for pt, eta in zip(pt, eta)]


def get_data(event, c_name, vars=['pt', 'm', 'eta', 'phi'], mask=None):
    """
    Extracts and processes data from a specified container within an event.

    event: The event object containing the data.
    c_name: Name of the container to get the data from.
    vars: List of the variables to pull from the container.
    mask: The mask to apply to the data, if applicable.
    max_objects: Maximum number of objects to retain after sorting and padding.
    return: Processed data from the specified container, after being sorted and padded.
    """
    data = []
    for var in vars:
        full_var_name = f"{c_name}_{var}"
        var_data = getattr(event, full_var_name)

        if mask is not None:
            masked_data = np.asarray([x for x, m in zip(var_data, mask) if m])
        else:
            masked_data = np.asarray([x for x in var_data]) # for some reason this syntax runs way faster!?

        data.append(masked_data)
    return data
    #return sort_and_pad(data, max_objects=max_objects)


def make_ntuples(tree, output_dir, use_TLA_triggers=True):
    if use_TLA_triggers == False:
        HLT_trig_names_ = [item for item in HLT_trig_names if 'TLA' not in item]
        L1_trig_names_ = [item for item in L1_trig_names if 'TLA' not in item]
    else:
        HLT_trig_names_ = HLT_trig_names
        L1_trig_names_ = L1_trig_names

    # Define lists to hold the data
    HLT_jet_list = []
    HLT_el_list = []
    HLT_muon_list = []
    HLT_ph_list = []
    MET_list = []
    pass_L1_unprescaled = []
    pass_HLT_unprescaled = []
    L1_muon_list = []
    L1_eFex_tau_list = []
    L1_MET_list = []
    L1_jFexSR_jet_list = []
    event_number_list = []
    run_number_list = []
    mu_list = []
    avg_int_list = []
    LB_list = []
    pass_single_jet_trigger = []
    
    
    # loop over events in the TTree and save the info to the lists
    for i, event in enumerate(tree):

        # Trigger Decisions
        trig_data = [str(trigger) for trigger in event.passedTriggers]
        pass_L1_unprescaled.append(1*any(trigger in L1_trig_names_ for trigger in trig_data)) # 0=no 1=yes
        pass_HLT_unprescaled.append(1*any(trigger in HLT_trig_names_ for trigger in trig_data)) # 0=no 1=yes
        pass_single_jet_trigger.append('HLT_j400_pf_ftf_preselj225_L1jJ180' in trig_data)

        # Event/run number
        event_number_list.append(event.eventNumber)
        run_number_list.append(event.runNumber)
        LB_list.append(event.lumiBlock)

        # Pileup
        mu_list.append(event.actualInteractionsPerCrossing)
        avg_int_list.append(event.averageInteractionsPerCrossing)
    
        # HLT Jets
        HLT_jet_data = get_data(event, 'HLT_jet', vars=['pt', 'eta', 'phi'])
        HLT_jet_list.append(sort_and_pad(HLT_jet_data, max_objects=6))

        # L1 SR Jets
        L1_jFexSR_jet_data = get_data(event, 'L1_jFexSRJet', vars=['et', 'eta', 'phi'])
        L1_jFexSR_jet_list.append(sort_and_pad(L1_jFexSR_jet_data, max_objects=6))
    
        # Electrons
        el_data = get_data(event, 'HLT_el', vars=['pt', 'eta', 'phi'])
        HLT_el_list.append(sort_and_pad(el_data, max_objects=3))
    
        # Muons
        muon_data = get_data(event, 'HLT_muon', vars=['pt', 'eta', 'phi'])
        HLT_muon_list.append(sort_and_pad(muon_data, max_objects=3))

        # L1 Muons
        L1_muon_data = get_data(event, 'L1Muon', vars=['et', 'eta', 'phi'])
        L1_muon_list.append(sort_and_pad(L1_muon_data, max_objects=4))
        
        # Photons
        ph_data = get_data(event, 'HLT_ph', vars=['pt', 'eta', 'phi'])
        HLT_ph_list.append(sort_and_pad(ph_data, max_objects=3))

        # L1 eTaus 
        L1_eFex_tau_data = get_data(event, 'L1Tau_eFex', vars=['et', 'eta', 'phi'])
        L1_eFex_tau_list.append(sort_and_pad(L1_eFex_tau_data, max_objects=4))
    

        # HLT MET
        MET_data = [np.float32(event.TrigMETMet), 0, np.float32(event.TrigMETMetPhi)]
        MET_list.append(MET_data)

        # L1 MET
        L1_MET_data = [np.float32(event.L1METMet), 0, np.float32(event.L1METMetPhi)]
        L1_MET_list.append(L1_MET_data)

    # Scale and fix the L1 data
    nmuon, nSRjet, netau = 4, 6, 4
    scale_factor = 10
    eta_factor = 40
    phi_factor = phi_res

    def fix_phi_range(phi_data):
        # Map values to [0,2pi] range
        phi_fixed = phi_data % (2*math.pi)
        # Scale to [0,128) range and round to nearest integer
        phi_scaled = np.round(phi_fixed * 128/(2*math.pi))
        
        # Take modulo 128 to handle edge cases
        phi_binned = phi_scaled % 128
        return phi_binned
    
    def load_and_scale(data, n_objects, scale_factor=10, eta_factor=40, phi_factor=phi_res):
        data[:, :, 0] *= scale_factor  # Scale the pT value
        data[:, :, 1] *= eta_factor  # Scale the angle value
        data[:, :, 2] = fix_phi_range(data[:, :, 2])
        return data.reshape(-1, 3 * n_objects)
        
    L1_jFexSR_jets = load_and_scale(np.array(L1_jFexSR_jet_list), nSRjet)
    L1_eFex_taus = load_and_scale(np.array(L1_eFex_tau_list), netau)
    L1_muons = load_and_scale(np.array(L1_muon_list), nmuon, scale_factor=10000)

    # Scale and fix the L1 MET
    L1_MET = np.array(L1_MET_list)
    L1_MET[:, 0] *= 10
    L1_MET[:, 2] = fix_phi_range(L1_MET[:, 2])
    L1_MET_fixed = np.zeros((L1_MET.shape[0], 2))
    L1_MET_fixed[:, 0] = L1_MET[:, 0]
    L1_MET_fixed[:, 1] = L1_MET[:, 2]
    L1_MET = L1_MET_fixed

    # Concatenate the data
    HLT_data = np.concatenate([np.array(HLT_jet_list), np.array(HLT_el_list), np.array(HLT_muon_list), np.array(HLT_ph_list), np.array(MET_list).reshape(-1, 1, 3)], axis=1)
    L1_data = np.concatenate([L1_jFexSR_jets, L1_eFex_taus, L1_muons, L1_MET], axis=1)
    L1_data = np.nan_to_num(L1_data, nan=0.0)
    
    # Flatten the HLT data (The L1 data is already flattened)
    HLT_num_features = HLT_data.shape[1] * HLT_data.shape[2]
    HLT_data = np.reshape(HLT_data, newshape=(-1, HLT_num_features))

    # create a dictionary of the data
    data_dict = {
        'HLT_data': HLT_data,
        'L1_data': L1_data,
        'pass_single_jet_trigger': np.array(pass_single_jet_trigger),
        'event_numbers': np.array(event_number_list),
        'run_numbers': np.array(run_number_list),
        'LBs': np.array(LB_list),
        'mus': np.array(mu_list),
        'avg_ints': np.array(avg_int_list),
        'pass_L1_unprescaled': np.array(pass_L1_unprescaled),
        'pass_HLT_unprescaled': np.array(pass_HLT_unprescaled),
    }

    # Get the current date in YYYYMMDD format
    current_date = datetime.now().strftime("%Y%m%d")

    # Construct the filename with the date and run number
    filename = f'{output_dir}/data_dict_{current_date}_{run_number_list[0]}.h5'

    with h5py.File(filename, 'w') as hf:
        for key, value in data_dict.items():
            hf.create_dataset(key, data=value)

    print(f'wrote to {filename}')

In [3]:
ttree_file = '/eos/home-m/mmcohen/ad_trigger_development/ops/data/trees/05-30-2025_ops_local_498335_lb201_-1/data-tree/data25_13p6TeV.00498335.physics_Main.merge.AOD.f1586_m2272.root'


# Open the ROOT file and get the tree
f = ROOT.TFile.Open(ttree_file)
td = f.Get('output_tree')
tree = td.Get("nominal")
nevents = tree.GetEntries()
print(f"Processing {nevents} events...")
if not tree:
    sys.stderr.write(f"Error: tree {args.tree} not found in {args.input_root}\n")
    sys.exit(1)

# Make the data
make_ntuples(tree, output_dir='/eos/home-m/mmcohen/ad_trigger_development/ops/data/ntuples')

Processing 16769 events...
wrote to /eos/home-m/mmcohen/ad_trigger_development/ops/data/ntuples/data_dict_20250623_498335.h5
